# Import Data

In [22]:
import numpy as np
from netCDF4 import Dataset
directory="/Users/lambda/Documents/Code/CaliforniaDrought2016/Data/"
#file=directory+"Pressure/hgt.mon.mean.nc"
#nc=Dataset(file,'r')
#pressure=nc.variables["hgt"][0:816,7,0:48,48:112]
file=directory+"Pressure/slp.mon.mean.nc"
nc=Dataset(file,'r')
#pressure=nc.variables["slp"][0:815,0:48,48:112]
pressure=nc.variables["slp"][0:816,0:48,48:112]
lat=nc.variables["lat"][:]
lon=nc.variables["lon"][:]
from numpy import loadtxt
CAMonthTotal= loadtxt(directory+'Precipitation/CAMonthTotal.txt')
ORMonthTotal= loadtxt(directory+'Precipitation/ORMonthTotal.txt')
WCMonthTotal= loadtxt(directory+'Precipitation/WCMonthTotal.txt')
CAMonthTotal=CAMonthTotal.reshape((816, 1))
ORMonthTotal=ORMonthTotal.reshape((816, 1))
WCMonthTotal=WCMonthTotal.reshape((816, 1))
CAMonthTotal=CAMonthTotal[0:816,:]
rlat=48
rlon=64

In [23]:
y_data=CAMonthTotal
x=pressure[0:3,:,:]
y=y_data[0:3,:]
for i in range(len(CAMonthTotal)/12):
    x=np.append(x,pressure[10+12*i:15+12*i,:,:],axis=0)
    y=np.append(y,y_data[10+12*i:15+12*i,:],axis=0)
l=len(x)
x_data=x.flatten()
y_data=y
    
x_data=np.reshape((x_data-np.mean(x_data))/np.std(x_data),(l,rlat,rlon,1))
y_data=(y_data-np.mean(y_data))/np.std(y_data)

# Set Computing Graph

### HyperParameters

In [24]:
ptraining=240
ctraining=[]
ctest=[]
#Convolution Layer Parameters
wc1=5
nc1=3
wc2=5 
nc2=1
wp1=2
wp2=2
fl=(rlat-wc1-wc2-wp1-wp2+4)*(rlon-wc1-wc2-wp1-wp2+4)*nc2
wfc=50
wfc2=10

### Graph

In [25]:
import tensorflow as tf
tf.reset_default_graph()
x = tf.placeholder('float', [None, rlat,rlon,1])
y = tf.placeholder('float',[None,1])
def convolutional_neural_network(x):
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='VALID')
    def maxpool2d(x,W):
        return tf.nn.max_pool(x, [1,W,W,1], strides=[1,1,1,1], padding='VALID')
    weights = {'W_conv1':tf.Variable(tf.random_normal([wc1,wc1,1,nc1],stddev=0.0001,name="W_conv1")),
               'W_conv2':tf.Variable(tf.random_normal([wc2,wc2,nc1,nc2],stddev=0.0001)),
               'W_fc':tf.Variable(tf.random_normal([fl,wfc],stddev=0.0001)),
               'W_fc2':tf.Variable(tf.random_normal([wfc,wfc2],stddev=0.0001)),
               'out':tf.Variable(tf.random_normal([wfc2, 1],stddev=0.0001))}

    biases = { 'b_conv1':tf.Variable(tf.random_normal([nc1],stddev=0.001)),
               'b_conv2':tf.Variable(tf.random_normal([nc2],stddev=0.001)),
               'b_fc':tf.Variable(tf.random_normal([wfc],stddev=0.001)),
               'b_fc2':tf.Variable(tf.random_normal([wfc2],stddev=0.001)),
               'out':tf.Variable(tf.random_normal([1],stddev=0.001))}
    
    
    conv1 = conv2d(x, weights['W_conv1']) + biases['b_conv1']
    pool1 = maxpool2d(conv1,wp1)

    conv2 = conv2d(pool1, weights['W_conv2']) + biases['b_conv2']
    pool2 = maxpool2d(conv2,wp1)

    fc = tf.reshape(pool2,[-1, fl])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc2 = tf.nn.relu(tf.matmul(fc, weights['W_fc2'])+biases['b_fc2'])
    output = tf.matmul(fc2, weights['out'])+biases['out'] 
    return output

In [26]:
prediction = convolutional_neural_network(x)
cost = tf.reduce_mean(tf.square(prediction-y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
all_saver = tf.train.Saver()

# Train

In [27]:
def train_neural_network(x):
    batch=120
    epoches=300
    gtestcost=np.inf
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(epoches):
            for b in range(ptraining/batch):
                epoch_x=x_data[b*batch:(b+1)*batch,:,:,:]
                epoch_y=y_data[b*batch:(b+1)*batch,:]
                sess.run(optimizer, feed_dict={x: epoch_x, y: epoch_y})
                trainingcost=sess.run(cost, feed_dict={x: epoch_x, y: epoch_y})
                ctraining.append(trainingcost)
            testcost=sess.run(cost, feed_dict={x: x_data[ptraining:,:,:,:], 
                                                   y: y_data[ptraining:,:]})
            print testcost
            if testcost < gtestcost and (testcost<0.25):
                gtestcost=testcost
                all_saver.save(sess, 'CNN_Trained')
            ctest.append(testcost)
        result=sess.run(prediction,feed_dict={x: x_data})
        sess.close()
    return gtestcost

In [28]:
tf.set_random_seed(23)
tempt=train_neural_network(x)

0.870992
0.871432
0.871888
0.872359
0.87285
0.873367
0.873893
0.874318
0.874353
0.873385
0.870539
0.864595
0.855217
0.842034
0.823251
0.803154
0.790592
0.78608
0.789634
0.796508
0.771663
0.744729
0.723194
0.686867
0.657704
0.625823
0.597734
0.57081
0.539593
0.498667
0.449699
0.394332
0.350509
0.32185
0.30984
0.32246
0.310252
0.294252
0.328199
0.283993
0.304871
0.274006
0.280626
0.27217
0.268965
0.280095
0.265465
0.284881
0.266749
0.281391
0.264362
0.270946
0.264624
0.270914
0.273826
0.274619
0.277322
0.274939
0.275451
0.276088
0.272604
0.277025
0.267246
0.272624
0.264705
0.268652
0.268438
0.264099
0.269609
0.264618
0.263235
0.27076
0.262605
0.256851
0.262399
0.270614
0.262208
0.25481
0.270252
0.300252
0.258401
0.244392
0.363656
0.243821
0.283318
0.249816
0.281386
0.256225
0.294528
0.264428
0.298459
0.267111
0.30321
0.271097
0.306631
0.277648
0.305193
0.285578
0.307303
0.303354
0.306587
0.315351
0.307571
0.333596
0.315518
0.335691
0.326324
0.328612
0.345875
0.332295
0.34236
0.347175
0.3

KeyboardInterrupt: 